# Solutions for in class exercise


In [ ]:
import os.path as op

import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from mne.datasets import somato

In [ ]:
# Load
data_path = somato.data_path()
subject = '01'
task = 'somato'
raw_fname = op.join(data_path, 'sub-{}'.format(subject), 'meg',
                    'sub-{}_task-{}_meg.fif'.format(subject, task))

In [ ]:
# Setup for reading the raw data
raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw, stim_channel='STI 014')

In [ ]:
# picks MEG channels
picks = mne.pick_types(raw.info, meg='grad', eeg=False, eog=True, stim=False)

In [ ]:
event_id, tmin, tmax = 1, -1., 3.  # Nice aspect of python to define variables thusly
baseline = (None, 0)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=baseline, reject=dict(grad=4000e-13, eog=350e-6),
                    preload=True)

epochs.resample(200., npad='auto')  # resample to reduce computation time

In [ ]:
# define frequencies of interest (log-spaced)
freqs = np.logspace(*np.log10([6, 35]), num=8)
n_cycles = freqs / 2.  # different number of cycle per frequency
power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, decim=3, n_jobs=1)

In [ ]:
bands = [(4, 8, 'Theta'), (8, 12, 'Alpha'),
                 (12, 30, 'Beta'), (30, 45, 'Gamma')]

In [ ]:
for i, j, k in bands:
    itc.plot_topomap(title=['Inter-Trial coherence in ' + k + ' band'], baseline=(-0.5, 0), mode='zscore', vmin=0.,
                 vmax=1.0, cmap='Reds', fmin=i, fmax=j, size=4)
